In [10]:
from profiler.discontinuity import detect_discontinuities
from pathlib import Path
from pyvideosync.data_pool import VideoFilesPool
from pyvideosync.videojson import Videojson
import json
import os
from typing import Sequence, Union, Optional, Dict, List

In [15]:
def find_all_drops_to_one(arr: Sequence[Union[int, float]]) -> List[Dict[str, Union[int, float]]]:
    """Find all points where a sequence value drops to 1 from a higher value.

    Scans through a 1-D sequence and detects each position `i` such that
    `arr[i] == 1` and `arr[i-1] > 1`. Returns a list of dictionaries containing
    the index where the drop occurs and the value immediately before the drop.

    Args:
        arr (Sequence[int or float]): A sequence (e.g., list or 1-D numpy array)
            of numeric values.

    Returns:
        List[Dict[str, Union[int, float]]]: A list of dicts, each with keys:
            - 'index' (int): The position in `arr` where the drop to 1 occurs.
            - 'prev_value' (int or float): The value at `arr[index - 1]`, i.e.
              the number that dropped down to 1.
        Returns an empty list if no drop-to-1 events are found.

    Examples:
        >>> find_all_drops_to_one([5, 4, 3, 1, 2, 1])
        [{'index': 3, 'prev_value': 3}, {'index': 5, 'prev_value': 2}]

        >>> find_all_drops_to_one([1, 2, 3, 4])
        []

        >>> import numpy as np
        >>> find_all_drops_to_one(np.array([10, 1, 1, 5, 1]))
        [{'index': 1, 'prev_value': 10}, {'index': 4, 'prev_value': 5}]
    """
    if not arr:
        return []
    drops: List[Dict[str, Union[int, float]]] = []
    for i in range(1, len(arr)):
        if arr[i] == 1 and arr[i - 1] > 1:
            drops.append({"index": i, "prev_value": arr[i - 1]})
    return drops

In [18]:
cam_recording_dir = "/mnt/datalake/data/TRD-43036/DBSTRD011/CAMERA/"
output_dir = "/home/auto/CODE/utils/video-sync/Testing/TRD011"
camera_serials = ["23512011"]

In [5]:
video_files_pool = VideoFilesPool()
for taskfolder in Path(cam_recording_dir).iterdir():
    if taskfolder.is_dir():
        for datefolder_path in taskfolder.iterdir():
            if datefolder_path.is_dir():
                for file_path in datefolder_path.iterdir():
                    video_files_pool.add_file(str(file_path.resolve()))

In [6]:
camera_files = video_files_pool.list_groups()

In [17]:
results = []
for camera_serial in camera_serials:
    for timestamp, cam_file_group in camera_files.items():
        json_files = [file for file in cam_file_group if file.endswith(".json")]
        json_path = json_files[0] if json_files else None
        if json_path is None:
            print(f"No JSON file found in group {timestamp}")
            continue

        videojson = Videojson(json_path)

        if not videojson.dic:
            continue
        chunk_serials = videojson.get_chunk_serial_list(camera_serial)
        frame_ids = videojson.get_frame_ids_list(camera_serial)

        chunk_drop_to_one = find_all_drops_to_one(chunk_serials)
        frame_drop_to_one = find_all_drops_to_one(frame_ids)

        if chunk_drop_to_one or frame_drop_to_one:
            results.append({
                "json": json_path,
                "chunk_drop_to_one": chunk_drop_to_one,
                "frame_drop_to_one": frame_drop_to_one,
            })

out_name = f"{camera_serial}_{timestamp}_drops.json"
out_path = os.path.join(output_dir, out_name)
with open(out_path, "w") as fp:
    json.dump(results, fp, indent=4)

print(f"Saved drop‐to‐1 results to {out_path}")

Find frame drop to 1! /mnt/datalake/data/TRD-43036/DBSTRD011/CAMERA/utsw_TRD011_day_1/20240716/utsw_TRD011_day_1_20240716_072715.json
[{'index': 29535, 'prev_value': 65535}]
Find frame drop to 1! /mnt/datalake/data/TRD-43036/DBSTRD011/CAMERA/utsw_TRD011_day_1/20240716/utsw_TRD011_day_1_20240716_080716.json
[{'index': 23070, 'prev_value': 65535}]
Find frame drop to 1! /mnt/datalake/data/TRD-43036/DBSTRD011/CAMERA/utsw_TRD011_day_1/20240716/utsw_TRD011_day_1_20240716_084718.json
[{'index': 16605, 'prev_value': 65535}]
Find frame drop to 1! /mnt/datalake/data/TRD-43036/DBSTRD011/CAMERA/utsw_TRD011_day_1/20240716/utsw_TRD011_day_1_20240716_092720.json
[{'index': 10140, 'prev_value': 65535}]
Find frame drop to 1! /mnt/datalake/data/TRD-43036/DBSTRD011/CAMERA/utsw_TRD011_day_1/20240716/utsw_TRD011_day_1_20240716_100722.json
[{'index': 3675, 'prev_value': 65535}]
Find frame drop to 1! /mnt/datalake/data/TRD-43036/DBSTRD011/CAMERA/utsw_TRD011_day_1/20240716/utsw_TRD011_day_1_20240716_102722.js